In [18]:
import os
import cv2
import numpy as np
from image_similarity_measures.quality_metrics import rmse, psnr, ssim, fsim, issm, sre, sam, uiq
from shutil import copyfile

# Paths
ct_path = "./example_CT_images/"
histology_path = "./example_Hist_images/"
paired_path = "./example_Paired_Images/"

# Ensure the paired images directory exists
if not os.path.exists(paired_path):
    os.makedirs(paired_path)

# Function to calculate all similarity measures
def calculate_similarity_metrics(img1, img2):
    # Ensure both images are (X, Y, 1)
    if len(img1.shape) == 2:
        img1 = np.expand_dims(img1, axis=-1)
    if len(img2.shape) == 2:
        img2 = np.expand_dims(img2, axis=-1)

    if img1.shape != img2.shape:
        raise ValueError(f"Image shapes are not the same: {img1.shape} vs {img2.shape}")

    print("Ready for metrics:", img1.shape, img2.shape)
    metrics = {
        'rmse': rmse(img1, img2),
        'psnr': psnr(img1, img2),
        'ssim': ssim(img1, img2),
        'fsim': fsim(img1, img2),
        'issm': issm(img1, img2),
        'sre': sre(img1, img2),
        'sam': sam(img1, img2),
        'uiq': uiq(img1, img2)
    }
    return metrics

# Function to find the best match
def find_best_match(ct_image, histology_images):
    best_match = None
    best_score = float('inf')
    best_metrics = None

    for hist_image_path in histology_images:
        hist_img = cv2.imread(hist_image_path)
        hist_img_gray = cv2.cvtColor(hist_img, cv2.COLOR_BGR2GRAY)
        resized_hist_img = cv2.resize(hist_img_gray, (ct_image.shape[1], ct_image.shape[0]))  # Resize to match CT image dimensions

        metrics = calculate_similarity_metrics(ct_image, resized_hist_img)
        avg_score = np.mean(list(metrics.values()))  # Average of all metric scores

        if avg_score < best_score:
            best_score = avg_score
            best_match = hist_image_path
            best_metrics = metrics

    return best_match, best_metrics

# Function to pair images
def pair_images(ct_images, histology_images):
    for ct_image_path in ct_images:
        ct_img = cv2.imread(ct_image_path, cv2.IMREAD_GRAYSCALE)
        if len(ct_img.shape) == 2:
            ct_img = np.expand_dims(ct_img, axis=-1)  # Ensure the image is (X, Y, 1)
        best_match, best_metrics = find_best_match(ct_img, histology_images)

        if best_match:
            # Copy the best matched images to the paired directory
            ct_image_name = os.path.basename(ct_image_path)
            hist_image_name = os.path.basename(best_match)

            paired_ct_path = os.path.join(paired_path, f"paired_ct_{ct_image_name}")
            paired_hist_path = os.path.join(paired_path, f"paired_hist_{hist_image_name}")

            copyfile(ct_image_path, paired_ct_path)
            copyfile(best_match, paired_hist_path)

            print(f"CT Image: {ct_image_name} is paired with Histology Image: {hist_image_name}")
            print(f"Metrics: {best_metrics}")
        else:
            print(f"No suitable match found for CT Image: {ct_image_path}")

# List all CT and Histology images
def list_images(ct_path, histology_path):
    ct_images = [os.path.join(ct_path, img) for img in os.listdir(ct_path) if img.endswith('.tif')]
    histology_images = [os.path.join(histology_path, img) for img in os.listdir(histology_path) if img.endswith('.jpg')]
    return ct_images, histology_images

# Execute the steps in the notebook
ct_images, histology_images = list_images(ct_path, histology_path)
pair_images(ct_images, histology_images)


Ready for metrics: (1592, 1576, 1) (1592, 1576, 1)


c:\Users\olive\AppData\Local\Programs\Python\Python310\lib\site-packages\image_similarity_measures\quality_metrics.py:171: RuntimeWarning: divide by zero encountered in log2
  return -np.sum(np.nan_to_num(H * np.log2(H)))
c:\Users\olive\AppData\Local\Programs\Python\Python310\lib\site-packages\image_similarity_measures\quality_metrics.py:171: RuntimeWarning: invalid value encountered in multiply
  return -np.sum(np.nan_to_num(H * np.log2(H)))
c:\Users\olive\AppData\Local\Programs\Python\Python310\lib\site-packages\image_similarity_measures\quality_metrics.py:188: RuntimeWarning: invalid value encountered in scalar divide
  return numerator / denominator
c:\Users\olive\AppData\Local\Programs\Python\Python310\lib\site-packages\image_similarity_measures\quality_metrics.py:297: RuntimeWarning: invalid value encountered in divide
  val = np.clip(numerator / denominator, -1, 1)
c:\Users\olive\AppData\Local\Programs\Python\Python310\lib\site-packages\image_similarity_measures\quality_metrics.

Ready for metrics: (1592, 1576, 1) (1592, 1576, 1)
Ready for metrics: (1592, 1576, 1) (1592, 1576, 1)
Ready for metrics: (1592, 1576, 1) (1592, 1576, 1)
Ready for metrics: (1592, 1576, 1) (1592, 1576, 1)
CT Image: Th24_PbRB-90kV-2-_rec00000338.tif is paired with Histology Image: ts19_EMA65_s05r01s05.jpg
Metrics: {'rmse': 0.04073031, 'psnr': 27.801644128952386, 'ssim': 0.5098097678117669, 'fsim': 0.28414267622880846, 'issm': 0.0, 'sre': 44.03528460258389, 'sam': 0.0, 'uiq': 0.00023106853096102102}
